In [ ]:
import sqlite3
import pandas as pd
import joblib
from collections import defaultdict, Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

# Conexión a la base de datos
conn = sqlite3.connect("db/market_place.db")

# Cargar recursos
productos = pd.read_sql_query("SELECT product_id, product_name FROM products", conn)
usuarios = pd.read_sql_query("SELECT user_id, segmento FROM users", conn)
rules = joblib.load("models/MBA/MBA_reglas_apriori.joblib")
tfidf_vectorizer = joblib.load("models/NLP/tfidf_vectorizer.joblib")
tfidf_matrix = joblib.load("models/NLP/tfidf_matrix.joblib")

# NearestNeighbors para NLP
nn_model = NearestNeighbors(n_neighbors=20, metric='cosine').fit(tfidf_matrix)

# Recomendaciones precalculadas de SVD
def recomendaciones_svd(user_id, top_n=10):
    query = """
    SELECT product_id, score FROM svd_predictions
    WHERE user_id = ?
    ORDER BY score DESC
    LIMIT ?
    """
    df = pd.read_sql_query(query, conn, params=(user_id, top_n))
    return df['product_id'].tolist()

# MBA rápido
def recomendaciones_mba(productos_en_carrito):
    recomendados = set()
    for producto in productos_en_carrito:
        sub_rules = rules[rules['antecedents'].apply(lambda x: producto in x)]
        recomendados.update(sub_rules['consequents'].explode().tolist())
    return list(recomendados)

# NLP rápido con NearestNeighbors
def recomendaciones_nlp(query, top_n=10):
    vector = tfidf_vectorizer.transform([query])
    indices = nn_model.kneighbors(vector, return_distance=False)[0][:top_n]
    return productos.iloc[indices]['product_id'].tolist()

# Motor híbrido explicativo
def products_recomender(user_id, productos_en_carrito=[], top_n=10):
    rec_svd = recomendaciones_svd(user_id, top_n * 2)
    rec_mba = recomendaciones_mba(productos_en_carrito)

    pesos = {'svd': 0.5, 'mba': 0.3, 'nlp': 0.2}
    explicacion = defaultdict(lambda: {'peso_total': 0, 'detalles': {}})

    # SVD
    for pid in rec_svd:
        score = conn.execute("SELECT score FROM svd_predictions WHERE user_id=? AND product_id=?", (user_id, pid)).fetchone()
        if score:
            explicacion[pid]['peso_total'] += pesos['svd']
            explicacion[pid]['detalles']['svd'] = round(score[0], 3)

    # MBA
    for pid in rec_mba:
        explicacion[pid]['peso_total'] += pesos['mba']
        explicacion[pid]['detalles']['mba'] = "Regla encontrada"

    # Armar resultados
    final = sorted(explicacion.items(), key=lambda x: x[1]['peso_total'], reverse=True)[:top_n]
    resultados = []
    for pid, data in final:
        nombre = productos[productos['product_id'] == pid]['product_name'].values[0]
        resultados.append({
            'product_id': pid,
            'product_name': nombre,
            'peso_total': round(data['peso_total'], 3),
            'detalles': data['detalles']
        })
    return resultados

In [ ]:
user = 6786

print("\n📦 Inicio (sin carrito, sin búsqueda):")
recomendaciones = products_recomender(user)
for r in recomendaciones:
    print(f"👉 {r['product_name']} (Sugerencia basada en perfil)")

print("\n🔍 Búsqueda: 'cookie'")
recomendaciones = recomendaciones_nlp(busqueda="cookie")
for r in recomendaciones:
    print(f"👉 {r['product_name']} (Coincidencia con búsqueda)")

print("\n🛒 Carrito contiene producto ID 22802 (chocolate chip cookies)")
recomendaciones = products_recomender(user, productos_en_carrito=[22802])
for r in recomendaciones:
    print(f"👉 {r['product_name']} (Peso: {r['peso_total']})")
    for fuente, valor in r['detalles'].items():
        print(f"   - {fuente.upper()}: {valor}")
